In [2]:
import pandas as pd

/var/folders/k_/x2cc2f4901v6lyz_t7w3w3h40000gn/T/ipykernel_11788/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data = pd.read_csv('./fraudTrain.csv')

In [4]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Dense, LeakyReLU
# from keras.optimizers import Adam
# from sklearn.utils import shuffle

# def build_generator(latent_dim):
#     model = Sequential()
#     model.add(Dense(32, input_dim=latent_dim))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(64))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(128))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(22, activation='tanh'))
#     return model

# def build_discriminator():
#     model = Sequential()
#     model.add(Dense(64, input_dim=30))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(32))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(1, activation='sigmoid'))
#     return model

# def build_gan(generator, discriminator):
#     discriminator.trainable = False
#     model = Sequential()
#     model.add(generator)
#     model.add(discriminator)
#     model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
#     return model

# data = pd.read_csv('./fraudTrain.csv')
# X_train, y_train = data.iloc[:, :-1].values, data.iloc[:, -1].values

# batch_size = 128
# epochs = 1000

# latent_dim = 10

# generator = build_generator(latent_dim)
# discriminator = build_discriminator()
# gan = build_gan(generator, discriminator)

# for epoch in range(epochs):
#     noise = np.random.normal(0, 1, (batch_size, latent_dim))
#     gen_samples = generator.predict(noise)
#     X = np.concatenate((X_train, gen_samples))
#     y = np.concatenate((y_train, np.zeros((batch_size, 1))))
#     X, y = shuffle(X, y)
#     discriminator.trainable = True
#     discriminator.train_on_batch(X, y)
#     noise = np.random.normal(0, 1, (batch_size, latent_dim))
#     y_gen = np.ones((batch_size, 1))
#     discriminator.trainable = False
#     gan.train_on_batch(noise, y_gen)

# noise = np.random.normal(0, 1, (batch_size, latent_dim))
# gen_samples = generator.predict(noise)

# X_augmented = np.concatenate((X_train, gen_samples))
# y_augmented = np.concatenate((y_train, np.ones((batch_size, 1))))

# X_augmented, y_augmented = shuffle(X_augmented, y_augmented)

In [5]:
data = data.drop(['Unnamed: 0', 'city_pop', 'job', 'street', 'gender', 'first', 'last', 'dob', 'unix_time', 'city', 'state', 'zip', 'category'], axis=1)

In [6]:
data['trans_date_trans_time'] = data['trans_date_trans_time'].apply(lambda x: pd.to_datetime(x))

In [7]:
data['customer_last_transaction_time'] = data.groupby('cc_num')['trans_date_trans_time'].transform('max')

In [8]:
data['merchant_last_transaction_time'] = data.groupby('merchant')['trans_date_trans_time'].transform('max')

In [9]:
data = data.sort_values(by='trans_date_trans_time')

data['last_customer_lat'] = data.groupby('cc_num')['lat'].transform('last')
data['last_customer_long'] = data.groupby('cc_num')['long'].transform('last')

data['last_merchant_lat'] = data.groupby('merchant')['lat'].transform('last')
data['last_merchant_long'] = data.groupby('merchant')['long'].transform('last')

In [10]:
import math

def calculate_distance(lat_prev, long_prev, lat_current, long_current):
    R = 6371  
    lat_prev_rad = math.radians(lat_prev)
    long_prev_rad = math.radians(long_prev)
    lat_current_rad = math.radians(lat_current)
    long_current_rad = math.radians(long_current)

    delta_lat = lat_current_rad - lat_prev_rad
    delta_long = long_current_rad - long_prev_rad

    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat_prev_rad) * math.cos(lat_current_rad) * math.sin(delta_long / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
data['customer_distance'] = data.apply(lambda x: calculate_distance(x['last_customer_lat'], x['last_customer_long'], x['lat'], x['long']), axis=1)
data['merchant_distance'] = data.apply(lambda x: calculate_distance(x['last_merchant_lat'], x['last_merchant_long'], x['merch_lat'], x['merch_long']), axis=1)

In [12]:
data['merch_score'] = data.groupby('merchant')['is_fraud'].transform('mean')
data['customer_score'] = data.groupby('cc_num')['is_fraud'].transform('mean')

In [27]:
from datetime import timedelta

data['customer_time_diff'] = (data['trans_date_trans_time'] - data['customer_last_transaction_time']).dt.total_seconds()
data['merchant_time_diff'] = (data['trans_date_trans_time'] - data['merchant_last_transaction_time']).dt.total_seconds()

In [28]:
data

,trans_date_trans_time,cc_num,merchant,amt,lat,long,trans_num,merch_lat,merch_long,is_fraud,...,last_customer_lat,last_customer_long,last_merchant_lat,last_merchant_long,customer_distance,merchant_distance,merch_score,customer_score,customer_time_diff,merchant_time_diff
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",4.97,36.0788,-81.1781,0b242abb623afc578575680df30655b9,36.011293,-82.048315,0,...,36.0788,-81.1781,30.4066,-91.1468,0.0,1050.310091,0.014207,0.000000,-46418877.0,-46417199.0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",107.23,48.8878,-118.2105,1f76529f8574734946361c461b024d99,49.159047,-118.186462,0,...,48.8878,-118.2105,40.6025,-80.3863,0.0,3089.902532,0.010787,0.000000,-46425509.0,-46428229.0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,220.11,42.1808,-112.2620,a1a22d70485983eac12b5b88dad1cf95,43.150704,-112.154481,0,...,42.1808,-112.2620,43.9446,-88.0911,0.0,1934.506583,0.002111,0.000000,-46336689.0,-46440126.0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",45.00,46.2306,-112.1138,6b849c168bdad6f867558c3793159a81,47.034331,-112.561071,0,...,46.2306,-112.1138,39.6967,-78.8031,0.0,2823.538028,0.003444,0.030426,-46230446.0,-46439540.0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,41.96,38.4207,-79.4629,a41d7549acf90789359a9aa5346dcb46,38.674999,-78.632459,0,...,38.4207,-79.4629,34.3470,-87.7154,0.0,942.996194,0.003769,0.000000,-46409545.0,-46432172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,15.56,37.7175,-112.4777,440b587732da4dc1a6395aba5fb41669,36.841266,-111.690765,0,...,37.7175,-112.4777,37.7175,-112.4777,0.0,119.752136,0.002622,0.005948,0.0,0.0
1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,51.70,39.2667,-77.5101,278000d2e0d2277d1de2f890067dcc0a,38.906881,-78.246528,0,...,39.2667,-77.5101,39.2667,-77.5101,0.0,75.104085,0.002284,0.015066,0.0,0.0
1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,105.93,32.9396,-105.8189,483f52fe67fabef353d552c1e662974c,33.619513,-105.130529,0,...,32.9396,-105.8189,32.9396,-105.8189,0.0,99.047734,0.001079,0.005314,0.0,0.0
1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",74.90,43.3526,-102.5411,d667cdcbadaaed3da3f4020e83591c83,42.788940,-103.241160,0,...,43.3526,-102.5411,43.3526,-102.5411,0.0,84.627652,0.002094,0.003953,0.0,0.0


In [14]:
merchant_history = data[['merchant', 'last_merchant_lat', 'last_merchant_long', 'merch_score', 'merchant_last_transaction_time']]
merchant_history = merchant_history.drop_duplicates()
customer_history = data[['cc_num', 'last_customer_lat', 'last_customer_long', 'customer_score', 'customer_last_transaction_time']]
customer_history = customer_history.rename(columns={'cc_num': 'merchant', 'last_customer_lat': 'last_merchant_lat', 'last_customer_long': 'last_merchant_long', 'customer_score': 'merch_score', 'customer_last_transaction_time': 'merchant_last_transaction_time'})
customer_history = customer_history.drop_duplicates()

combined_history = pd.concat([merchant_history, customer_history])
combined_history = combined_history.drop_duplicates()

In [15]:
combined_history.to_csv('combined_history.csv', index=False)

In [17]:
new = data[['customer_score', 'merch_score', 'customer_distance', 'merchant_distance', 'customer_time_diff', 'merchant_time_diff', 'is_fraud']]

In [18]:
new.to_csv('new.csv', index=False)

In [19]:
data = data.drop(['cc_num', 'merchant', 'trans_num', 'trans_date_trans_time', 'last_merch_lat', 'last_merchant_long', 'last_customer_lat', 'last_customer_long'], axis = 1)

In [20]:
data.to_csv('preprocessed_fraudTrain.csv')

In [30]:
from imblearn.over_sampling import SMOTE

X = new.iloc[:, :-1].values
y = new.iloc[:, -1].values

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=0)

classifier = RandomForestClassifier(n_estimators=100, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99    258036
           1       0.98      0.99      0.99    257632

    accuracy                           0.99    515668
   macro avg       0.99      0.99      0.99    515668
weighted avg       0.99      0.99      0.99    515668



In [35]:
import pickle

with open('fraud_detection_model.pkl', 'wb') as file:
    pickle.dump(classifier, file)